In [2]:
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np

In [3]:
app = dash.Dash()

In [4]:
query = """
SELECT DISTINCT
ORDER_ID_ANON,
SUBSTR(CAST(CAST(ORDER_ID_ANON AS INT64) AS STRING), 1, 9) AS SERIAL,
MSISDN_ANON,
ORDER_STATUS_DESC,
ACTION_STATUS_DESC,
ACTION_TYPE_DESC 
FROM `bcx-insights.telkom_customerexperience.orders_20190903_00_anon`
GROUP BY ORDER_ID_ANON,SERIAL, MSISDN_ANON,ORDER_STATUS_DESC,ACTION_STATUS_DESC,ACTION_TYPE_DESC
ORDER BY ORDER_ID_ANON,SERIAL, MSISDN_ANON
LIMIT 1000
"""

In [5]:
df = pd.read_gbq(query,
                 project_id='bcx-insights',
                 dialect='standard')

In [6]:
A = list(df["ACTION_TYPE_DESC"].unique())
B = list(df["SERIAL"].unique())
node_list = list(set(A+B))
G = nx.Graph()
for i in node_list:
    G.add_node(i)

In [7]:
for i,j in df.iterrows():
    G.add_edges_from([(j["ACTION_TYPE_DESC"],j["SERIAL"])])

In [8]:
pos = nx.spring_layout(G, k=0.5, iterations=50)
for n, p in pos.items():
    G.node[n]['pos'] = p

In [9]:
##num_nodes = 2000
##edge = df['itemset'][:num_nodes]
#create graph G
##G = nx.Graph()
#G.add_nodes_from(node)
##G.add_edges_from(edge)
#get a x,y position for each node
##pos = nx.layout.spring_layout(G)

In [10]:
#Create Edges
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

In [11]:
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),  
        line=dict(width=2)))
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [12]:
#add color to node points
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = 'Name: ' + str(adjacencies[0]) + '<br># of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])

In [13]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

In [14]:
#iplot(fig)
#plotly.plot(fig)

In [15]:
app.layout = html.Div([
    html.Div([html.H1("Network Graph")], className="row", style={'textAlign': "center"}),
    html.Div([dcc.Graph(id="my-graph", figure = fig)]),
], className="container")

In [16]:
if __name__ == '__main__':
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Debugger PIN: 150-607-500
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

